In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score,f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
dataset_obuka = pd.read_csv('https://raw.githubusercontent.com/mvikic/speaker-gender-recognition/master/baza_neutral.csv')
dataset_obuka.head()

ModuleNotFoundError: No module named 'google'

In [ ]:
classes = dataset_obuka.iloc[:,-3]
print(classes.unique())

classes_svi = classes.unique()
print(classes_svi)

classes_izbaci = ['p0006', 'p0005', 'p0001', 'p0002', 'p1007', 'p1008', 'p1009', 'p1060', 'p1010', 'p1011', 'p1012']
classes_ostavi = [i for i in classes_svi if i not in classes_izbaci]

print(classes_ostavi)
print(len(classes_ostavi))

In [ ]:
dataset_bezImena = dataset_obuka.drop(['name', 'age'], axis = 1)
dataset_bezImena.head()
print(dataset_bezImena.shape)

In [ ]:
dataset_bezImena_train = dataset_bezImena
for i in range(0, len(classes_izbaci)):    
    indexes_izb = dataset_bezImena_train.loc[dataset_bezImena_train['class'].str.find(classes_izbaci[i]) != -1].index
    dataset_bezImena_train.drop(indexes_izb, axis = 0, inplace = True)
dataset_train = dataset_bezImena_train

print(dataset_train.shape)

In [ ]:
dataset_train = dataset_train.drop(['class'], axis = 1)
dataset_train.head()

In [ ]:
X_obuka= dataset_train.iloc[:, :-1]
y_obuka = dataset_train.iloc[:, -1]

X_obuka.head()

In [ ]:
#Zenski i muski

print('Nedostajućih vrijednosti ima: ', X_obuka.isnull().sum().sum())
print('Oznake klasa su: ', y_obuka.unique())
pd.set_option('display.max_columns', None)
dataset_train.describe()

In [ ]:
#Muski uzorci
male = dataset_train[dataset_train['gender'] == 'M']
male_sum = sum(dataset_train['gender'] == 'M')
print('Klasi MALE pripada: ',male_sum, ' uzoraka')
pd.set_option('display.max_columns', None)
male.describe()

In [ ]:
#Zenski uzorci
female = dataset_train[dataset_train['gender'] == 'F']
female_sum = sum(dataset_train['gender'] == 'F')
print('Klasi FEMALE: ',female_sum, ' uzoraka')
pd.set_option('display.max_columns', None)
female.describe()

In [ ]:
# #ODABIR PARAMETARA ZA SVM METODU SA STANDARDIZOVANIM VREDNOSTIMA NA CELOM SKUPU - FOR PETLJA
labels_y = ['M', 'F']
s = StandardScaler()
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
acc = []
iteracija = 0 
for c in [1,10,100]:
     for F in ['linear', 'rbf']:
       for d in ['ovo', 'ovr']:
            indexes = kf.split(X_obuka, y_obuka)
            acc_tmp = []
            fin_conf_mat = np.zeros((len(np.unique(y_obuka)),len(np.unique(y_obuka))))
            for train_index, test_index in indexes:
                iteracija += 1
                pd.set_option('display.max_rows', None)
                #print(X_obuka.iloc[train_index,:].dtypes)
                classifier = SVC(C=c, kernel=F,decision_function_shape=d)  
                X_std1 = s.fit_transform(X_obuka.iloc[train_index,:])
                X_std_test = s.transform(X_obuka.iloc[test_index,:])
                classifier.fit(X_std1, y_obuka.iloc[train_index])
                #classifier.fit(X_obuka[train_index, :], y_train.iloc[train_index])
                y_pred = classifier.predict(X_std_test)
                acc_tmp.append(accuracy_score(y_obuka.iloc[test_index], y_pred))
                fin_conf_mat += confusion_matrix(y_obuka.iloc[test_index], y_pred, labels=labels_y)
            print(int(iteracija/10))
            print('za parametre C=', c, ', kernel=', F, ' i pristup=',d, ' tacnost je: ', np.mean(acc_tmp),
            ' a mat. konf. je:')
            print(fin_conf_mat)
            acc.append(np.mean(acc_tmp))
print('najbolja tacnost je u iteraciji broj: ', np.argmax(acc))

In [ ]:
dataset_bezImena_test = dataset_obuka.drop(['name', 'age'], axis = 1)
indeksi_svi = []
for i in range(0, len(classes_ostavi)):    
    indexes_ost = dataset_bezImena_test.loc[dataset_bezImena_test['class'].str.find(classes_ostavi[i]) != -1].index
    indeksi_svi.append(indexes_ost)
    dataset_bezImena_test.drop(indexes_ost, axis = 0, inplace = True)
dataset_test = dataset_bezImena_test

print(dataset_test.shape)
dataset_test.head()

In [ ]:
dataset_test = dataset_test.drop(['class'], axis = 1)
dataset_test.head()

In [ ]:
def evaluation_classif(conf_mat):
    TP = conf_mat[1,1]
    FP = conf_mat[0,1]
    FN = conf_mat[1,0]
    TN = conf_mat[0,0]
    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    F_score = 2*precision*sensitivity/(precision+sensitivity)
    print('precision: ', precision)
    print('accuracy: ', accuracy)
    print('sensitivity/recall: ', sensitivity)
    print('specificity: ', specificity)
    print('F score: ', F_score)

In [ ]:
X_test= dataset_test.iloc[:, :-1]
y_test = dataset_test.iloc[:, -1]

print(X_test.shape)
X_test.head()

In [ ]:
labels_y = ['M', 'F']
conf_mat_svm = np.zeros((len(np.unique(y_obuka)),len(np.unique(y_obuka))))

s = StandardScaler()
#s.fit(X_obuka)
X_obuka_std = s.fit_transform(X_obuka)
X_test_std = s.fit_transform(X_test)


classifier_kon = SVC(C=10, kernel='linear', decision_function_shape='ovr')
classifier_kon.fit(X_obuka_std, y_obuka)
   
# testiranje
y_pred_kon = classifier_kon.predict(X_test_std) 

conf_mat_svm = confusion_matrix(y_test, y_pred_kon, labels=labels_y)

print(conf_mat_svm)

evaluation_classif(conf_mat_svm)

In [ ]:
"""labels_y = ['M', 'F']
classifier = SVC(C=10, kernel='linear', decision_function_shape='ovr')
classifier.fit(X_obuka, y_obuka)
# testiranje
y_pred = classifier.predict(X_test) 
conf_mat = confusion_matrix(y_test, y_pred, labels=labels_y)
print(conf_mat)                                               
evaluation_classif(conf_mat)"""

In [ ]:
#PCA
pca = PCA(n_components=None)
pca.fit(X_obuka_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
X_obuka_pca = pca.transform(X_obuka_std)
X_test_pca = pca.transform(X_test_std)
pca_dimenzije = pca.n_components_
print('Redukovani prostor ima dimenziju: ', pca_dimenzije)

In [ ]:
#PCA
labels_y = ['M', 'F']
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
acc = []
iteracija = 0 
for c in [1,10,100]:
     for F in ['linear', 'rbf']:
       for d in ['ovo', 'ovr']:
          for p in [125,150,175,200]:
              indexes = kf.split(X_obuka, y_obuka)
              acc_tmp = []
              fin_conf_mat = np.zeros((len(np.unique(y_obuka)),len(np.unique(y_obuka))))
              for train_index, test_index in indexes:
                      iteracija += 1
                      X_std1 = s.fit_transform(X_obuka.iloc[train_index,:])
                      X_std_test = s.transform(X_obuka.iloc[test_index,:])
                      pca = PCA(n_components=p)
                      pca.fit(X_std1)
                      X_train_r = pca.transform(X_std1)
                      X_test_r=pca.transform(X_std_test)
                      classifier = SVC(C=c, kernel=F,decision_function_shape=d)
                      classifier.fit(X_train_r, y_obuka.iloc[train_index])
                      y_pred = classifier.predict(X_test_r)
                      acc_tmp.append(accuracy_score(y_obuka.iloc[test_index], y_pred))
                      fin_conf_mat += confusion_matrix(y_obuka.iloc[test_index], y_pred, labels=labels_y)
              print(int(iteracija/10))
              print('za parametre n_components=',p , ' C=', c, ', kernel=', F, ' i pristup=',d, ' tacnost je: ', np.mean(acc_tmp),
              ' a mat. konf. je:')
              print(fin_conf_mat)
              acc.append(np.mean(acc_tmp))
print('najbolja tacnost je u iteraciji broj: ', np.argmax(acc))

In [ ]:
labels_y = ['M', 'F']
pca_conf_mat_svc = np.zeros((len(np.unique(y_obuka)),len(np.unique(y_obuka))))

s = StandardScaler()
#s.fit(X_obuka)
X_obuka_std = s.fit_transform(X_obuka)
X_test_std = s.fit_transform(X_test)

pca = PCA(n_components=200)
pca.fit(X_obuka_std)
X_obuka_PCA = pca.transform(X_obuka_std)
X_test_PCA = pca.transform(X_test_std)


classifier_kon = SVC(C=10, kernel='linear', decision_function_shape='ovr')
classifier_kon.fit(X_obuka_PCA, y_obuka)
   
# testiranje
y_pred_kon = classifier_kon.predict(X_test_PCA) 

pca_conf_mat_svc = confusion_matrix(y_test, y_pred_kon, labels=labels_y)

print(pca_conf_mat_svc)

evaluation_classif(pca_conf_mat_svc)

In [ ]:
#LDA
labels_y = ['M', 'F']
lda_conf_mat_svm = np.zeros((len(np.unique(y_obuka)),len(np.unique(y_obuka))))

s = StandardScaler()
#s.fit(X_obuka)
X_obuka_std = s.fit_transform(X_obuka)
X_test_std = s.transform(X_test)

lda = LinearDiscriminantAnalysis(n_components=1)
lda.fit(X_obuka_std, y_obuka)
X_obuka_LDA = lda.transform(X_obuka_std)
X_test_LDA = lda.transform(X_test_std)

classifier_kon = SVC(C=10, kernel='linear', decision_function_shape='ovr')
classifier_kon.fit(X_obuka_LDA, y_obuka)
   
# testiranje
y_pred_kon = classifier_kon.predict(X_test_LDA) 

lda_conf_mat_svm = confusion_matrix(y_test, y_pred_kon, labels=labels_y)

print(lda_conf_mat_svm)

evaluation_classif(lda_conf_mat_svm)